<center>
# ПРИЛОЖЕНИЕ 5
<center>
# РЕШЕНИЕ ДОМАШНЕГО ЗАДАНИЯ 2

In [1]:
# загружаем необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [2]:
# считываем данные
data = pd.read_csv('Data/cs-training.csv', encoding='cp1251', sep=',')

In [3]:
data.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
# смотрим типы переменных и информацию о количестве пропусков
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
Unnamed: 0                              150000 non-null int64
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null int64
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(8)
memory usage: 13.7 MB


In [5]:
# присваиваем верный тип переменной SeriousDlqin2yrs
data['SeriousDlqin2yrs'] = data['SeriousDlqin2yrs'].astype('str')

In [6]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
data.describe()

,Unnamed: 0,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000,150000.000,150000.000,150000.000,150000.000,120269.000,150000.000,150000.000,150000.000,150000.000,146076.000
mean,75000.500,6.048,52.295,0.421,353.005,6670.221,8.453,0.266,1.018,0.240,0.757
std,43301.415,249.755,14.772,4.193,2037.819,14384.674,5.146,4.169,1.130,4.155,1.115
min,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,37500.750,0.030,41.000,0.000,0.175,3400.000,5.000,0.000,0.000,0.000,0.000
50%,75000.500,0.154,52.000,0.000,0.367,5400.000,8.000,0.000,1.000,0.000,0.000
75%,112500.250,0.559,63.000,0.000,0.868,8249.000,11.000,0.000,2.000,0.000,1.000
max,150000.000,50708.000,109.000,98.000,329664.000,3008750.000,58.000,98.000,54.000,98.000,20.000


По-видимому, переменная `Unnamed: 0` является идентификатором (количество непропущенных наблюдений совпадает с максимальным значением). Переменная `RevolvingUtilizationOfUnsecuredLines` имеет значения больше 1. У переменной `age` есть значения меньше 18 лет и больше 70 лет (строгого ограничения по верхней границе возраста для желающих оформить кредит у банков в Штатах нет, но некоторые банки устанавливают планку в 70 лет). У переменных `NumberOfTime30-59DaysPastDueNotWorse`, `NumberOfTime60-89DaysPastDueNotWorse`, `NumberOfTimes90DaysLate`, характеризирующих глубину просрочки, есть большое значение 98, которое скорее всего является служебным кодом, а не количеством просрочек. Переменная `DebtRatio` имеет значения больше 1. Переменная `MonthlyIncome` имеет аномально низкое минимальное значение (0), тогда как в Штатах кредитуют при минимальном ежегодном доходе не менее 15000 $.

In [7]:
# ко всем значениям переменной DebtRatio больше 1, 
# но меньше 100000 добавляем 0.0
data['DebtRatio'] = data['DebtRatio'].map(
    lambda x: float('0.0' + ''.join(str(x).split('.'))) if x > 1 and x < 100000 else x)

# ко всем значениям переменной DebtRatio 
# больше 100000 добавляем 0.0
data['DebtRatio'] = data['DebtRatio'].map(
    lambda x: float('0.' + ''.join(str(x).split('.'))) if x >= 100000 else x)

In [8]:
# выводим 10 минимальных и максимальных значений
# переменной MonthlyIncome
print(data['MonthlyIncome'].nsmallest(10))
print(data['MonthlyIncome'].nlargest(10))

14    0.000
50    0.000
73    0.000
90    0.000
298   0.000
326   0.000
508   0.000
537   0.000
605   0.000
629   0.000
Name: MonthlyIncome, dtype: float64
73763    3008750.000
137140   1794060.000
111365   1560100.000
50640    1072500.000
122543    835040.000
123291    730483.000
93564     702500.000
96549     699530.000
119136    649587.000
37078     629000.000
Name: MonthlyIncome, dtype: float64


In [9]:
# очень низкие и очень высокие значения переменной 
# MonthlyIncome записываем как пропуски
data['MonthlyIncome'] = np.where((data['MonthlyIncome'] < 1000) | (data['MonthlyIncome'] > 1000000), 
                                 np.NaN, data['MonthlyIncome'])

In [10]:
# удаляем идентификатор Unnamed
data.drop('Unnamed: 0', axis=1, inplace=True)

In [11]:
# очень низкие и очень высокие значения переменной 
# age записываем как пропуски
data['age'] = np.where((data['age'] < 21) | (data['age'] > 80), np.NaN, data['age'])

In [12]:
# создаем переменную Ratio - отношение количества 
# просрочек 90+ к общему количеству просрочек
a = 0.01
data['Ratio'] = (data['NumberOfTimes90DaysLate'] + a) / (
    data['NumberOfTimes90DaysLate'] + data['NumberOfTime30-59DaysPastDueNotWorse'] + \
    data['NumberOfTime60-89DaysPastDueNotWorse']) + a
data['Ratio'].replace([np.inf, -np.inf], 0, inplace=True)

In [13]:
# создаем индикаторы нулевых значений переменных NumberOfTimes90DaysLate 
# и RevolvingUtilizationOfUnsecuredLines_is_0
data['NumberOfTimes90DaysLate_is_0'] = np.where(
    data['NumberOfTimes90DaysLate'] == 0, 1, 0).astype('object')
data['RevolvingUtilizationOfUnsecuredLines_is_0'] = np.where(
    data['RevolvingUtilizationOfUnsecuredLines'] == 0, 1, 0).astype('object')

In [14]:
# смотрим частоты некоторых переменных
for i in ['NumberOfTime30-59DaysPastDueNotWorse', 
          'NumberOfTime60-89DaysPastDueNotWorse',
          'NumberOfTimes90DaysLate',
          'NumberOfDependents']:
    print(data[i].value_counts(dropna=False))

0     126018
1      16033
2       4598
3       1754
4        747
5        342
98       264
6        140
7         54
8         25
9         12
96         5
10         4
12         2
13         1
11         1
Name: NumberOfTime30-59DaysPastDueNotWorse, dtype: int64
0     142396
1       5731
2       1118
3        318
98       264
4        105
5         34
6         16
7          9
96         5
8          2
11         1
9          1
Name: NumberOfTime60-89DaysPastDueNotWorse, dtype: int64
0     141662
1       5243
2       1555
3        667
4        291
98       264
5        131
6         80
7         38
8         21
9         19
10         8
11         5
96         5
13         4
12         2
14         2
15         2
17         1
Name: NumberOfTimes90DaysLate, dtype: int64
0.000     86902
1.000     26316
2.000     19522
3.000      9483
nan        3924
4.000      2862
5.000       746
6.000       158
7.000        51
8.000        24
9.000         5
10.000        5
13.000        1
20.000    

In [15]:
# преобразовываем переменные в категориальные, выделив
# значения 96 и 98 в отдельную категорию и значения
# больше 3 также в отдельную категорию
data.at[data['NumberOfTime30-59DaysPastDueNotWorse'].isin(
    [96, 98]), 'NumberOfTime30-59DaysPastDueNotWorse'] = -1
data.loc[data['NumberOfTime30-59DaysPastDueNotWorse'] > 3, 'NumberOfTime30-59DaysPastDueNotWorse'] = 'more_3'

data.at[data['NumberOfTime60-89DaysPastDueNotWorse'].isin(
    [96, 98]), 'NumberOfTime60-89DaysPastDueNotWorse'] = -1
data.loc[data['NumberOfTime60-89DaysPastDueNotWorse'] > 3, 'NumberOfTime60-89DaysPastDueNotWorse'] = 'more_3'

data.at[data['NumberOfTimes90DaysLate'].isin(
    [96, 98]), 'NumberOfTimes90DaysLate'] = -1
data.loc[data['NumberOfTimes90DaysLate'] > 3, 'NumberOfTimes90DaysLate'] = 'more_3'

In [16]:
# снова смотрим переменные
for i in ['NumberOfTime30-59DaysPastDueNotWorse', 
          'NumberOfTime60-89DaysPastDueNotWorse',
          'NumberOfTimes90DaysLate',
          'NumberOfDependents']:
    print(data[i].value_counts())

0         126018
1          16033
2           4598
3           1754
more_3      1328
-1           269
Name: NumberOfTime30-59DaysPastDueNotWorse, dtype: int64
0         142396
1           5731
2           1118
3            318
-1           269
more_3       168
Name: NumberOfTime60-89DaysPastDueNotWorse, dtype: int64
0         141662
1           5243
2           1555
3            667
more_3       604
-1           269
Name: NumberOfTimes90DaysLate, dtype: int64
0.000     86902
1.000     26316
2.000     19522
3.000      9483
4.000      2862
5.000       746
6.000       158
7.000        51
8.000        24
9.000         5
10.000        5
13.000        1
20.000        1
Name: NumberOfDependents, dtype: int64


In [17]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
train, test, y_train, y_test = train_test_split(data.drop('SeriousDlqin2yrs', axis=1), 
                                                data['SeriousDlqin2yrs'], 
                                                test_size=.3, 
                                                stratify=data['SeriousDlqin2yrs'], 
                                                random_state=100)

In [18]:
def importance_auc(train, target, test):
    train_copy = train.copy()
    test_copy = test.copy()
    # создаем список переменных
    col_list = train_copy.select_dtypes(include=['number']).columns
    # создаем список, в который будем записывать auc
    auc_list = []
    for i in col_list:
        train_copy[i].fillna(train_copy[i].median(), inplace=True)
        power = PowerTransformer(
            method='yeo-johnson', standardize=True).fit(train_copy[i].values.reshape(-1,1))
        train_copy[i] = power.transform(train_copy[i].values.reshape(-1,1))
        test_copy[i].fillna(test_copy[i].median(), inplace=True)
        test_copy[i] = power.transform(test_copy[i].values.reshape(-1,1))
        logreg = LogisticRegression(solver='liblinear').fit(train_copy[i].values.reshape(-1,1), y_train)
        auc = roc_auc_score(y_test, logreg.predict_proba(test_copy[i].values.reshape(-1,1))[:, 1])
        auc_list.append(auc)
    
    result = pd.DataFrame({'Переменная': col_list,
                           'AUC': auc_list})
    result = np.round(result.sort_values(by='AUC', ascending=False), 3)
    cm = sns.light_palette('yellow', as_cmap=True)
    return(result.style.background_gradient(cmap=cm))

In [19]:
importance_auc(train, y_train, test)

,Переменная,AUC
0,RevolvingUtilizationOfUnsecuredLines,0.774
7,Ratio,0.764
1,age,0.632
3,MonthlyIncome,0.585
4,NumberOfOpenCreditLinesAndLoans,0.554
6,NumberOfDependents,0.546
5,NumberRealEstateLoansOrLines,0.542
2,DebtRatio,0.533


In [20]:
def diagnostics_skewness(df, target):
    # создаем список переменных
    col_list = df.select_dtypes(include=['number']).columns
    skew_initial_list = []
    skew_pos_reciprocal_list = []
    skew_neg_reciprocal_list = []
    skew_log_list = []
    skew_corr_log_001_list = []
    skew_corr_log_01_list = []
    skew_corr_log_1_list = []
    skew_corr_log_5_list = []
    skew_cbrt_list = []
    skew_sqrt_list = []
    
    
    # запускаем цикл, который вычисляет скос по каждой 
    # преобразованной переменной
    for i in col_list:
        a = 0.001
        skew_initial = df[i].skew()
        skew_pos_reciprocal = (1 / (df[i].clip(0) + a)).skew()
        skew_neg_reciprocal = (-1 / (df[i].clip(0) + a)).skew()
        skew_log = np.log(df[i].clip(0) + a).skew()
        skew_corr_log_001 = np.log((df[i].clip(0) / df[i].mean()) + 0.001).skew()
        skew_corr_log_01 = np.log((df[i].clip(0) / df[i].mean()) + 0.01).skew()
        skew_corr_log_1 = np.log((df[i].clip(0) / df[i].mean()) + 0.1).skew()
        skew_corr_log_5 = np.log((df[i].clip(0) / df[i].mean()) + 0.5).skew()
        skew_cbrt = np.cbrt(df[i]).skew()
        skew_sqrt = np.sqrt(df[i]).skew()
        skew_initial_list.append(skew_initial)
        skew_pos_reciprocal_list.append(skew_pos_reciprocal)
        skew_neg_reciprocal_list.append(skew_neg_reciprocal)
        skew_log_list.append(skew_log)
        skew_corr_log_001_list.append(skew_corr_log_001)
        skew_corr_log_01_list.append(skew_corr_log_01)
        skew_corr_log_1_list.append(skew_corr_log_1)
        skew_corr_log_5_list.append(skew_corr_log_5)
        skew_cbrt_list.append(skew_cbrt)
        skew_sqrt_list.append(skew_sqrt)   

    result = pd.DataFrame({'Переменная': col_list,
                           'Skew_init': skew_initial_list,
                           'Skew_pos_recip': skew_pos_reciprocal_list,
                           'Skew_neg_recip': skew_neg_reciprocal_list,
                           'Skew_log': skew_log_list,
                           'Skew_adj_log (k=0.001)': skew_corr_log_001_list,
                           'Skew_adj_log (k=0.01)': skew_corr_log_01_list,
                           'Skew_adj_log (k=0.1)': skew_corr_log_1_list,
                           'Skew_adj_log (k=0.5)': skew_corr_log_5_list,
                           'Skew_cbrt': skew_cbrt_list,
                           'Skew_sqrt': skew_sqrt_list})
    result = np.round(result.sort_values(by='Skew_init', ascending=False), 3)
    cm = sns.light_palette('magenta', as_cmap=True)
    return(result.style.background_gradient(cmap=cm))

In [21]:
diagnostics_skewness(train, y_train)

,Переменная,Skew_init,Skew_pos_recip,Skew_neg_recip,Skew_log,Skew_adj_log (k=0.001),Skew_adj_log (k=0.01),Skew_adj_log (k=0.1),Skew_adj_log (k=0.5),Skew_cbrt,Skew_sqrt
0,RevolvingUtilizationOfUnsecuredLines,111.487,2.914,-2.914,-0.655,-0.14,0.948,7.015,21.195,21.281,41.136
3,MonthlyIncome,39.839,1.925,-1.925,0.154,0.157,0.186,0.427,1.099,1.732,3.93
7,Ratio,4.912,-1.489,1.489,2.213,1.734,1.998,2.736,3.545,3.005,3.868
5,NumberRealEstateLoansOrLines,3.691,0.522,-0.522,-0.489,-0.488,-0.451,-0.315,0.002,-0.286,0.065
6,NumberOfDependents,1.609,-0.384,0.384,0.406,0.405,0.426,0.498,0.656,0.529,0.709
4,NumberOfOpenCreditLinesAndLoans,1.238,8.767,-8.767,-5.045,-3.783,-2.159,-0.808,-0.073,-0.841,-0.027
2,DebtRatio,0.981,4.856,-4.856,-0.977,-1.365,-0.762,-0.259,0.13,-0.233,0.157
1,age,-0.008,1.205,-1.205,-0.561,-0.561,-0.554,-0.499,-0.352,-0.368,-0.274


In [22]:
# выполняем импутацию пропусков в переменной MonthlyIncome медианами
train['age'].fillna(train['age'].median(), inplace=True)
test['age'].fillna(train['age'].median(), inplace=True)

train['MonthlyIncome'].fillna(train['MonthlyIncome'].median(), inplace=True)
test['MonthlyIncome'].fillna(train['MonthlyIncome'].median(), inplace=True)

In [23]:
# пропуски в переменной NumberOfDependents заменяем нулем, 
# значения больше 4 выделяем в отдельную категорию
train.at[train['NumberOfDependents'].isnull(), 'NumberOfDependents'] = 0
train.loc[train['NumberOfDependents'] > 4, 'NumberOfDependents'] = 'more_4'

test.at[test['NumberOfDependents'].isnull(), 'NumberOfDependents'] = 0
test.loc[test['NumberOfDependents'] > 4, 'NumberOfDependents'] = 'more_4'

In [24]:
# все значения переменной RevolvingUtilizationOfUnsecuredLines
# больше 2 пометим как пропуски и заменим медианой
train['RevolvingUtilizationOfUnsecuredLines'] = np.where(train['RevolvingUtilizationOfUnsecuredLines'] > 2, 
                                                         np.NaN, train['RevolvingUtilizationOfUnsecuredLines'])
test['RevolvingUtilizationOfUnsecuredLines'] = np.where(test['RevolvingUtilizationOfUnsecuredLines'] > 2, 
                                                        np.NaN, test['RevolvingUtilizationOfUnsecuredLines'])

train['RevolvingUtilizationOfUnsecuredLines'].fillna(
    train['RevolvingUtilizationOfUnsecuredLines'].median(), inplace=True)
test['RevolvingUtilizationOfUnsecuredLines'].fillna(
    train['RevolvingUtilizationOfUnsecuredLines'].median(), inplace=True)

In [25]:
categorical_columns = train.dtypes[train.dtypes == 'object'].index
for c in categorical_columns:
    print(train[c].value_counts(dropna=False))

0         88243
1         11242
2          3228
3          1189
more_3      934
-1          164
Name: NumberOfTime30-59DaysPastDueNotWorse, dtype: int64
0         99165
1          3661
2          1119
3           477
more_3      414
-1          164
Name: NumberOfTimes90DaysLate, dtype: int64
0         99670
1          4041
2           777
3           227
-1          164
more_3      121
Name: NumberOfTime60-89DaysPastDueNotWorse, dtype: int64
0.0       63479
1.0       18427
2.0       13712
3.0        6640
4.0        2045
more_4      697
Name: NumberOfDependents, dtype: int64
1    99165
0     5835
Name: NumberOfTimes90DaysLate_is_0, dtype: int64
0    97496
1     7504
Name: RevolvingUtilizationOfUnsecuredLines_is_0, dtype: int64


In [26]:
num_columns = ['MonthlyIncome', 'NumberRealEstateLoansOrLines', 'NumberOfOpenCreditLinesAndLoans']
# вычисляем лямбды для выполнения преобразования
power = PowerTransformer(method='yeo-johnson', standardize=False).fit(train[num_columns])
# применяем преобразование Йео-Джонсона к переменным
# в обучающем и тестовом наборах
train[num_columns] = power.transform(train[num_columns])
test[num_columns] = power.transform(test[num_columns])

In [27]:
train['DebtRatio'] = np.log((train['DebtRatio'] / train['DebtRatio'].mean()) + 0.5)
test['DebtRatio'] = np.log((test['DebtRatio'] / test['DebtRatio'].mean()) + 0.5)

In [28]:
# пишем функцию, вычисляющую IV по всем 
# количественным предикторам
def numeric_IV(df, target): 
    # создаем список, в который будем записывать IV
    iv_list = []
    # записываем константу, которую будем добавлять,
    # чтобы избежать деления на 0
    a= 0.0001
    # задаем зависимую переменную
    target = target.astype('str')
    # создаем копию датафрейма
    df = df.copy()
    # отбираем количественные переменные
    numerical_columns = df.select_dtypes(include=['number']).columns
    # запускаем цикл, который вычисляет IV по каждой 
    # выбранной переменной
    for var_name in numerical_columns:
        # разбиваем переменную на 10 квантилей
        df[var_name] = pd.qcut(df[var_name].values, 10, duplicates='drop').codes
        # строим таблицу сопряженности между категоризированной 
        # переменной и зависимой переменной
        biv = pd.crosstab(df[var_name], target)        
        # вычисляем IV на основе таблицы сопряженности
        IV = sum(((biv['0'] / sum(biv['0']) + a) - 
                  (biv['1'] / sum(biv['1']) + a)) * np.log(
            (biv['0'] / sum(biv['0']) + a) / (biv['1'] / sum(biv['1']) + a)))
        # добавляем вычисленное IV в список, где хранятся IV
        iv_list.append(IV)
    # создаем список с названиями столбцов
    col_list = list(numerical_columns)
    # создаем датафрейм с двумя столбцами, в одном - названия переменных,
    # в другом - IV этих переменных
    result = pd.DataFrame({'Название переменной' : col_list,'IV' : iv_list})
    # добавляем дополнительный столбец "Полезность", задаем строковые значения,
    # которые будут выводится в зависимости от величины IV
    result['Полезность'] = ['Подозрительно высокая' if x > 0.5 else 'Сильная' 
                            if x <= 0.5 and x > 0.3 else 'Средняя'
                            if x <= 0.3 and x > 0.1 else 'Слабая' 
                            if x <= 0.1 and x > 0.02 else 'Бесполезная' 
                            for x in result['IV']]  # по Наиму Сиддики
    # возвращаем датафрейм, отсортированный по убыванию IV
    return(result.sort_values(by='IV', ascending=False))

In [29]:
numeric_IV(train, y_train)

,Название переменной,IV,Полезность
6,Ratio,1.316,Подозрительно высокая
0,RevolvingUtilizationOfUnsecuredLines,1.106,Подозрительно высокая
1,age,0.225,Средняя
3,MonthlyIncome,0.077,Слабая
4,NumberOfOpenCreditLinesAndLoans,0.058,Слабая
2,DebtRatio,0.046,Слабая
5,NumberRealEstateLoansOrLines,0.013,Бесполезная


In [30]:
# пишем функцию, выполняющую биннинг
def user_bin(df, target, number):    
    # увеличиваем максимальную ширину столбца
    pd.set_option('max_colwidth', 800) 
    # задаем список, где будут хранится IV
    iv_list = []
    # задаем список, где будет хранится информация о количестве бинов
    bins_list = [] 
    # задаем список, где будет хранится информация о бинах
    groups_list = []  
    # записываем константу, которую будем добавлять,
    # чтобы избежать деления на 0
    a = 0.0001
    # задаем зависимую переменную
    target = target.astype('str')
    # отбираем столбцы, у которых больше 10 уникальных значений
    df_subset = df.loc[:, df.apply(pd.Series.nunique) > 10]
    # из этих столбцов отбираем только количественные
    numerical_columns = df_subset.select_dtypes(include=['number']).columns    
    # запускаем цикл, который вычисляет IV по каждой 
    # выбранной переменной
    for var_name in numerical_columns:  
        # фиксируем количество бинов
        num = number
        # создаем точки разбиения
        bins = np.linspace(df_subset[var_name].min(), df_subset[var_name].max(), num) 
        # округляем значения точек разбиения
        rounded_bins = np.round(bins, 2)
        # создаем бины
        groups = np.digitize(df_subset[var_name], rounded_bins)
        # строим таблицу сопряженности между категоризированной 
        # переменной и зависимой переменной
        biv = pd.crosstab(groups, target)
        # вычисляем IV на основе таблицы сопряженности        
        IV = sum(((biv['0'] / sum(biv['0']) + a) - 
                  (biv['1'] / sum(biv['1']) + a)) * np.log(
            (biv['0'] / sum(biv['0']) + a) / (biv['1'] / sum(biv['1']) + a)))
        # добавляем вычисленное IV в список, где хранятся IV
        iv_list.append(IV) 
        # добавляем информацию о количестве бинов в список, где хранится
        # информация о количестве бинов
        bins_list.append(num)
        # добавляем бины в список, где хранится 
        # информация о бинах
        groups_list.append(rounded_bins)
    # создаем список с названиями столбцов    
    col_list = list(numerical_columns) 
    # создаем датафрейм с четырьмя столбцами, в первом - названия переменных,
    # во втором - бины, в третьем - IV, в четвертом - количество бинов
    result = pd.DataFrame({'Переменная' : col_list, 
                           'Бины': groups_list, 
                           'IV' : iv_list,
                           'Количество_бинов' : bins_list})    
    # возвращаем датафрейм, отсортированный по убыванию IV
    return(result.sort_values(by='IV', ascending=False))

In [31]:
user_bin(train, y_train, 6)

,Переменная,Бины,IV,Количество_бинов
0,RevolvingUtilizationOfUnsecuredLines,"[0.0, 0.4, 0.8, 1.2, 1.6, 2.0]",1.016,6
6,Ratio,"[0.0, 0.2, 0.41, 0.61, 0.82, 1.02]",0.795,6
1,age,"[21.0, 32.8, 44.6, 56.4, 68.2, 80.0]",0.212,6
5,NumberRealEstateLoansOrLines,"[0.0, 0.51, 1.01, 1.52, 2.03, 2.53]",0.061,6
4,NumberOfOpenCreditLinesAndLoans,"[0.0, 1.86, 3.72, 5.58, 7.44, 9.3]",0.054,6
3,MonthlyIncome,"[5.74, 6.51, 7.28, 8.05, 8.81, 9.58]",0.048,6
2,DebtRatio,"[-0.69, -0.25, 0.2, 0.65, 1.09, 1.54]",0.044,6


In [32]:
bins = [-np.inf,   0.51, 1.01, 1.52, 2.03, np.inf]
train['linescat'] = pd.cut(train['NumberRealEstateLoansOrLines'], bins).astype('object')
test['linescat'] = pd.cut(test['NumberRealEstateLoansOrLines'], bins).astype('object')

In [33]:
bins = [-np.inf, -0.25, 0.2, 0.65, 1.09, np.inf]
train['debtcat'] = pd.cut(train['DebtRatio'], bins).astype('object')
test['debtcat'] = pd.cut(test['DebtRatio'], bins).astype('object')

In [34]:
# выполняем стандартизацию
num_cols = [c for c in train.columns if train[c].dtype.name != 'object']
train_copy = train.copy()
for i in num_cols:
    train[i] = (train[i] - train[i].mean()) / train[i].std()
    test[i] = (test[i] - train_copy[i].mean()) / train_copy[i].std()

In [35]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105000 entries, 49898 to 65332
Data columns (total 15 columns):
RevolvingUtilizationOfUnsecuredLines         105000 non-null float64
age                                          105000 non-null float64
NumberOfTime30-59DaysPastDueNotWorse         105000 non-null object
DebtRatio                                    105000 non-null float64
MonthlyIncome                                105000 non-null float64
NumberOfOpenCreditLinesAndLoans              105000 non-null float64
NumberOfTimes90DaysLate                      105000 non-null object
NumberRealEstateLoansOrLines                 105000 non-null float64
NumberOfTime60-89DaysPastDueNotWorse         105000 non-null object
NumberOfDependents                           105000 non-null object
Ratio                                        105000 non-null float64
NumberOfTimes90DaysLate_is_0                 105000 non-null object
RevolvingUtilizationOfUnsecuredLines_is_0    105000 non-null ob

In [36]:
# выполняем дамми-кодирование
X_train = pd.get_dummies(train)
X_test = pd.get_dummies(test)

In [37]:
# строим модель логистической регрессии
logreg = LogisticRegression(solver='lbfgs', max_iter=500).fit(X_train, y_train)
print("AUC на обучающей выборке: {:.3f}".
    format(roc_auc_score(y_train, logreg.predict_proba(X_train)[:, 1])))
print("AUC на контрольной выборке: {:.3f}".
    format(roc_auc_score(y_test, logreg.predict_proba(X_test)[:, 1])))

AUC на обучающей выборке: 0.862
AUC на контрольной выборке: 0.861
